In [ ]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import Range1d
output_notebook()

import numpy as np


from voltagebudget import util
from voltagebudget import neurons
from voltagebudget.util import locate_firsts
from voltagebudget.util import locate_peaks
from voltagebudget.util import filter_voltages
from fakespikes import util as fsutil

# Define input

In [ ]:
t = 0.4
stim_number = 40
stim_onset = 0.2
stim_offset = 0.25
stim_rate = 8
dt = 1e-5
seed_stim = 1
ns_x, ts_x = util.poisson_impulse(
        t,
        stim_onset,
        stim_offset - stim_onset,
        stim_rate,
        n=stim_number,
        dt=dt,
        seed=None)

print(">>> {} spikes".format(ts_x.size))
print(">>> {} population rate".format(ts_x.size / (stim_offset - stim_onset)))

In [ ]:
p = figure(plot_width=400, plot_height=200)
p.circle(ts_x, ns_x, color="black")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'N'
p.x_range = Range1d(0, t)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

- Shadow voltage response of an LIF neuron to the input

In [ ]:
#set mode
mode = 'regular'
params, w_in, bias, sigma = util.read_modes(mode)
bias = np.asarray(bias)

In [ ]:
print(w_in, bias, sigma)

In [ ]:
# -
M = 20
A = .2e-9
f = 8
b_in = neurons.shadow_adex(M, t, ns_x, ts_x, f=f, A=A, w_in=w_in, bias_in=bias-A/2, sigma=sigma, **params)
times = b_in['times']
v_in = b_in['V_m_thresh']

In [ ]:
# -
p = figure(plot_width=400, plot_height=200)
for n in range(10):
    p.line(x=times, y=v_in[n, :], color="black", alpha=0.1)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
combine = False
ns_p, ts_p = locate_peaks(b_in, stim_onset, stim_offset, combine=combine)
# -
p = figure(plot_width=400, plot_height=200)
for n in range(10):
    p.line(x=times, y=v_in[n, :], color="black", alpha=0.1)
    if not combine:
        p.circle(x=ts_p[n], y=v_in[n, :].max(), color="purple")
    else:
        p.line(x=ts_p[0], y=np.linspace(v_in[:, :].min(), v_in[:, :].max(), 10), color="purple")
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

# Budget analysis

In [ ]:
budget_y = b_in

In [ ]:
times = budget_y['times']

p = figure(plot_width=400, plot_height=200)
for n in range(M):
    v = budget_y['V_m'][n, :]
    p.line(x=times, y=v, color="black", alpha=0.1)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
times = budget_y['times']

p = figure(plot_width=400, plot_height=200)
for n in range(M):
    v = budget_y['V_comp'][n, :]
    p.line(x=times, y=v, color="black", alpha=0.1)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
show(p)

In [ ]:
times = budget_y['times']

Et = -52e-3
p = figure(title="{}".format(A), plot_width=400, plot_height=230)
for n in range(M):
    v = budget_y['V_osc'][n, :] 
    p.line(x=times, y=v, color="black", alpha=0.1)
p.xaxis.axis_label = 'Time (s)'
p.yaxis.axis_label = 'Vm (volts)'
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
# p.y_range = Range1d(0.05, 0.08)
show(p)